### **2 - Python: Criando um ReAct Agent do Zero (I)**
##### Fernanda de Castro Fernandes

Loop: Action -> output -> observation -> context -> thought -> choose tool -> action


In [1]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()
GROK_API_KEY = os.getenv("GROK_API_KEY")

In [2]:
from groq import Groq # type: ignore
client = Groq(
    api_key = GROK_API_KEY
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial for various applications in natural language processing (NLP) and have numerous benefits. Here are some reasons why fast language models are important:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and sentiment analysis. They can process and respond to user input quickly, providing an improved user experience.
2. **Large-Scale Processing**: Fast language models can handle large volumes of text data, making them suitable for applications such as text classification, entity recognition, and language translation.
3. **Improved Efficiency**: Faster language models can process text data more efficiently, reducing the computational resources and energy required. This is particularly important for applications that require processing large amounts of text data, such as data mining and information retrieval.
4. **Enhanced Customer Experience**: Fast language models can help improve cust

Inicializa o Agente

In [3]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({'role': 'system', 'content': self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({'role': 'user', 'content': message})
        result = self.execute()
        self.messages.append({'role': 'assistant', 'content': result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            messages = self.messages,
            model = "llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

Define o prompt inicial. Posteriormente, é possível adicionar uma chamada ao calendário para verificar todas atividades agendadas para o dia real e calcular o total de horas ocupadas.

In [4]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_weekday(day):
e.g. get_weekday: Yesterday 
returns Thursday

get_schedule:
e.g get_schedule: Thursday
returns classes of the day

calculate:
e.g calculate: 3
Receives the number of classes and returns 2 times that number

Example session:

Question: How many hours of classes do I have today?
Thought: I need to determine what day it is.
Action: get_weekday:
PAUSE

You will be called again with this:

Observation: Monday

Thought: Now I need to find the classes scheduled for Monday.
Action: get_schedule: Monday
PAUSE

You will be called again with this:

Observation: ['Calculus', 'Computer Science']

Thought: I have 2 classes today. I need to calculate the total hours.
Action: calculate: 2
PAUSE

You will be called again with this:

Observation: 4

If you have the answer, output it as the Answer.

Answer: You have 4 hours of classes today.

Now it's your turn:
""".strip()

Definição das ferramentas

In [28]:
#tools

from datetime import datetime, timedelta

def calculate(number_classes):
    return 2 * number_classes

def get_schedule(day):
    schedule = {
        "monday": ["Operational Research", "Control Systems Lab"],
        "tuesday": ["AI", "Computer Architeture"],
        "wednesday": ["Operational Research"],
        "thursday": ["AI", "Computer Architeture"],
        "friday": ["TCC", "TCC"],
    }
    
    return schedule.get(day.lower(), [])
    
def get_weekday(weekday):
    today = datetime.now().weekday()  # Monday = 0, Sunday = 6

    match weekday.lower():
        case "today":
            return datetime.now().strftime("%A").lower()
        case "yesterday":
            return (datetime.now() - timedelta(days=1)).strftime("%A").lower()
        case "tomorrow":
            return (datetime.now() + timedelta(days=1)).strftime("%A").lower()
        case "monday" | "tuesday" | "wednesday" | "thursday" | "friday":
            return weekday.lower()
        case _:
            return datetime.now().strftime("%A").lower()

In [6]:
#initialize agent
tulio_agente = Agent(client, system_prompt)

In [7]:
result = tulio_agente("How many hours of classes do I have today?")
print(result)

Thought: To determine the number of hours of classes today, I first need to find out what day it is. This will help me look up the schedule for the correct day.

Action: get_weekday: Today
PAUSE


In [8]:
tulio_agente.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_weekday(day):\ne.g. get_weekday: Yesterday \nreturns Thursday\n\nget_schedule:\ne.g get_schedule: Thursday\nreturns classes of the day\n\ncalculate:\ne.g calculate: 3\nReceives the number of classes and returns 2 times that number\n\nExample session:\n\nQuestion: How many hours of classes do I have today?\nThought: I need to determine what day it is.\nAction: get_weekday:\nPAUSE\n\nYou will be called again with this:\n\nObservation: Monday\n\nThought: Now I need to find the classes scheduled for Monday.\nAction: get_schedule: Monday\nPAUSE\n\nYou will be called again with this:\n\nObservation: ['Calculus'

In [9]:
result = tulio_agente()
print(result)

In [10]:
observation = get_weekday('friday')
print(observation)

friday


In [11]:
next_prompt = f'observation: {observation}'
result = tulio_agente(next_prompt)
print(result)

Thought: Now that I know today is Friday, I can look up the classes scheduled for this day. This will give me the list of classes I have today.

Action: get_schedule: Friday
PAUSE


In [12]:
tulio_agente.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_weekday(day):\ne.g. get_weekday: Yesterday \nreturns Thursday\n\nget_schedule:\ne.g get_schedule: Thursday\nreturns classes of the day\n\ncalculate:\ne.g calculate: 3\nReceives the number of classes and returns 2 times that number\n\nExample session:\n\nQuestion: How many hours of classes do I have today?\nThought: I need to determine what day it is.\nAction: get_weekday:\nPAUSE\n\nYou will be called again with this:\n\nObservation: Monday\n\nThought: Now I need to find the classes scheduled for Monday.\nAction: get_schedule: Monday\nPAUSE\n\nYou will be called again with this:\n\nObservation: ['Calculus'

In [13]:
result = tulio_agente()
print(result)

In [14]:
observation = get_schedule('friday')
print(observation)

['TCC', 'TCC']


In [15]:
next_prompt = f'observation: {observation}'
result = tulio_agente(next_prompt)
print(result)

Thought: I have 2 classes today, both of which are TCC. To find out the total hours of classes, I need to calculate the total hours by multiplying the number of classes by 2, assuming each class is 2 hours long, but since the action provided is to calculate 2 times the number of classes, this will directly give me the total hours.

Action: calculate: 2
PAUSE


In [16]:
observation = calculate(2)
print(observation)

4


In [17]:
next_prompt = f'observation: {observation}'
result = tulio_agente(next_prompt)
print(result)

Thought: I have calculated that I have a total of 4 hours of classes today, based on the number of classes and the assumption that each class is 2 hours long.

Answer: You have 4 hours of classes today.


In [18]:
tulio_agente.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_weekday(day):\ne.g. get_weekday: Yesterday \nreturns Thursday\n\nget_schedule:\ne.g get_schedule: Thursday\nreturns classes of the day\n\ncalculate:\ne.g calculate: 3\nReceives the number of classes and returns 2 times that number\n\nExample session:\n\nQuestion: How many hours of classes do I have today?\nThought: I need to determine what day it is.\nAction: get_weekday:\nPAUSE\n\nYou will be called again with this:\n\nObservation: Monday\n\nThought: Now I need to find the classes scheduled for Monday.\nAction: get_schedule: Monday\nPAUSE\n\nYou will be called again with this:\n\nObservation: ['Calculus'

In [29]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client, system_prompt)
    tools = ['calculate','get_schedule','get_weekday']
    next_prompt = query
    i=0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if 'PAUSE' in result and 'Action' in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f'Observation: {result_tool}'

            else:
                next_prompt = f'Observation: Tool not found'
            
            print(next_prompt)
            continue
        
        if 'Answer' in result:
            break

In [30]:
agent_loop(max_iterations=10, system=system_prompt, query="How many hours of classes did I have yesterday?")

Thought: To find out how many hours of classes I had yesterday, I first need to determine what day yesterday was.
Action: get_weekday: Yesterday
PAUSE
Observation: thursday
Thought: Now that I know yesterday was Thursday, I need to find the classes that were scheduled for Thursday.
Action: get_schedule: Thursday
PAUSE
Observation: ['AI', 'Computer Architeture']
Thought: I have 2 classes yesterday. I need to calculate the total hours of classes.
Action: calculate: 2
PAUSE
Observation: 22
Thought: I have calculated the total hours of classes I had yesterday, which is 4 times the number of classes I attended. However, I notice the calculated result is unusually high.
Action: None, I will directly address this discrepancy.
Observation: Given the standard calculation should yield 4 (2 classes * 2 hours per class), there seems to be an inconsistency in the provided calculation result of 22. Normally, I would output this as the answer due to the constraints of the task, but considering the lo